**Agenda/goal**: To build an MCP server that identifies and ranks the best locations to open a business.

**Install necessary packages**

In [ ]:
!pip install tavily-python

**Import packages**

In [ ]:
from tavily import TavilyClient
from openai import OpenAI
import json
import os

**Import API keys**

In [ ]:
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
os.environ["TAVILY_API_KEY"] = "TAVILY_API_KEY"

**Set API keys**

In [ ]:
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

**Define a tool to extact factual location intelligence using Tavily search results combined with OpenAI-based extraction**


In [ ]:
def extract_cities_with_llm(enriched_data, country):
    """
    OpenAI extracts ONLY real cities explicitly mentioned in Tavily results.
    Global, non-US restricted.
    """

    prompt = f"""
You are a factual extraction system.

TASK:
Extract only the names of real, officially recognized cities located in the country: {country}.

STRICT RULES:
- DO NOT hallucinate.
- A city must appear explicitly and verbatim in the provided text (title, snippet, url, or content).
- Preserve all Unicode characters exactly as they appear (e.g., “Ålesund”, “Tromsø”, “Düsseldorf”).
- Remove duplicates.
- If the text mentions a region, county, island, district, mountain, airport, or non-city area, EXCLUDE it. Examples to exclude: “Lofoten”, “Nordland”, “Sicily”, “Kyushu”.
- If the extracted name is NOT a real city in {country}, exclude it even if it appears in text.
- If the spelling appears truncated or incomplete, exclude it. Example: exclude “Stevanga” because it is not a valid city.
- If uncertain, exclude it.

OUTPUT FORMAT:
Output ONLY valid JSON:
{{"cities": ["CityA", "CityB"]}}


DATA:
{json.dumps(enriched_data, indent=2)}

Now output only JSON:
"""

    resp = openai_client.responses.create(
        model="gpt-4.1",
        input=prompt,
        max_output_tokens=300
    )

    try:
        return json.loads(resp.output_text).get("cities", [])
    except:
        return []


def fetch_factual_location_data(location: str, business_type: str) -> dict:
    """
    Performs factual location intelligence using Tavily search results combined with
    OpenAI-based extraction. This tool supports two modes—country-level discovery of
    suitable cities for a business, and city-level retrieval of factual market data.

    The function NEVER fabricates locations. All extracted cities must appear explicitly
    within Tavily search result text (title, snippet, content), as enforced by the
    `extract_cities_with_llm` function.

    Args:
        location (str):
            A country name (e.g., "Italy") or a specific city (e.g., "Milan, Italy").
            - If only one segment is provided (no comma), the function assumes **country mode**.
            - If a comma-separated value is provided, it assumes **city mode**.

        business_type (str):
            The type of business under investigation (e.g., "coffee shop", "bakery").
            This will be included in the search query to keep results contextually relevant.

    Behavior:
        • COUNTRY MODE
            - Tavily is queried for “best cities in <country> for opening a <business_type>”.
            - Raw content, snippets, and URLs are retrieved.
            - OpenAI is used strictly to extract real cities mentioned in the data.
            - Returns a structured list of candidate cities + citations.

        • CITY MODE
            - Tavily is queried for factual market data related to the business.
            - Returns raw factual content + citations for further processing.

    Returns:
        dict:
            A structured dictionary depending on the mode.

            COUNTRY MODE example:
            {
                "mode": "country",
                "input": "Italy",
                "business_type": "bakery",
                "candidate_cities": ["Rome", "Milan", "Lecce"],
                "citations": ["url1", "url2", ...]
            }

            CITY MODE example:
            {
                "mode": "city",
                "input": "Milan, Italy",
                "business_type": "bakery",
                "facts": ["raw factual text ...", "raw text ..."],
                "citations": ["url1", "url2", ...]
            }

    Notes:
        • No hallucinated cities are ever added—only cities physically appearing in Tavily content.
        • Useful as the first step in a pipeline for ranking cities or producing recommendations.
        • This function provides **raw factual scaffolding**, not opinions or assumptions.
    """

    is_country = len(location.split(",")) == 1

    # ---------- COUNTRY MODE ----------
    if is_country:
        query = f"Best cities in {location} for opening a {business_type}. Factual sources only."

        tv = tavily.search(
            query=query,
            include_raw_content=True,
            max_results=12
        )

        enriched = []
        for r in tv.get("results", []):
            enriched.append({
                "title": r.get("title"),
                "url": r.get("url"),
                "snippet": r.get("snippet"),
                "content": r.get("content")
            })

        cities = extract_cities_with_llm(enriched, location)

        return {
            "mode": "country",
            "input": location,
            "business_type": business_type,
            "candidate_cities": cities,
            "citations": [r.get("url") for r in tv.get("results", [])]
        }

    # ---------- CITY MODE ----------
    else:
        query = f"Market data for opening a {business_type} in {location}. Factual only."

        tv = tavily.search(
            query=query,
            include_raw_content=True,
            max_results=10
        )

        return {
            "mode": "city",
            "input": location,
            "business_type": business_type,
            "facts": [r.get("content") for r in tv.get("results", [])],
            "citations": [r.get("url") for r in tv.get("results", [])]
        }


**Evaluates and ranks a list of cities within a country based on the factual business potential for launching a specific type of business.**

In [ ]:
def extract_city_market_signals(city, business_type, enriched_data):
    """
    Uses OpenAI to extract ONLY factual signals about:
    - demand
    - foot traffic
    - tourism
    - business activity
    - growth
    - culture related to the business
    - economic attractiveness

    It MUST NOT invent numbers or events.
    It can only summarize what appears in data.
    """

    prompt = f"""
You are a factual business analysis system.

GOAL:
Extract factual signals that influence opening a **{business_type}** in **{city}**.

RULES:
- Only use information explicitly present in the text.
- DO NOT create or guess data.
- If something is unclear or missing, do NOT assume it.
- Summarize signals such as:
    • foot traffic
    • tourism
    • business growth
    • demand indicators
    • cultural fit
    • economic activity
- NO numerical values unless they exist in the text.
- Output must be JSON with fields:
{{
  "signals": ["fact1", "fact2"],
  "score_reason": "short explanation based only on facts"
}}

DATA:
{json.dumps(enriched_data, indent=2)}
"""

    resp = openai_client.responses.create(
        model="gpt-4.1-mini",
        input=prompt,
        max_output_tokens=400
    )

    try:
        return json.loads(resp.output_text)
    except:
        return {
            "signals": [],
            "score_reason": "No signals extracted."
        }


def score_city_from_signals(signals_list):
    """
    Score is simply count of useful signals (0–10 scale).
    More factual signals = higher opportunity.
    """

    score = len(signals_list)

    # Cap at 10
    if score > 10:
        score = 10

    return score


def rank_cities_market_potential(country:str, business_type:str, candidate_cities:list[str]) -> list[dict]:
    """
    Evaluates and ranks a list of cities within a country based on the factual business
    potential for launching a specific type of business. This tool runs Tavily searches
    for each city, extracts factual business-related signals using OpenAI, converts the
    signals into an objective numeric score, and produces a ranked list.

    This tool does NOT guess or hallucinate business conditions. Its scoring and reasoning
    rely strictly on signals found explicitly inside Tavily search results.

    Args:
        country (str):
            The country where the evaluation is taking place.
            Used to contextualize search queries and avoid misinterpreting cities with
            similar names across different regions.

        business_type (str):
            The type of business being evaluated (e.g., "coffee shop", "bakery").
            This keyword shapes the Tavily search queries and the extraction logic.

        candidate_cities (list[str]):
            A list of cities previously verified to physically exist in the country.
            These typically come from `fetch_factual_location_data()`.

    Behavior:
        For each city:
            1. Tavily is queried for factual signals related to:
               - business demand
               - foot traffic
               - tourism
               - startup or small-business activity
               - density/competition
               - economic or demographic indicators

            2. Search results are aggregated into a structured dataset.

            3. `extract_city_market_signals()` is used to pull ONLY factual signals from
               the data (no opinions, no assumptions, no invented metrics).

            4. `score_city_from_signals()` transforms those signals into a **0–10 score**
               based on presence, frequency, and strength of evidence.

        Finally:
            • All cities are sorted in descending score order.
            • A structured, machine-readable list is returned.

    Returns:
        list[dict]:
            A list of ranked city entries, each containing:

            {
                "city": "Milan",
                "score": 7,
                "signals": [
                    "Tourism in Milan is growing steadily.",
                    "High density of cafes in the central district.",
                    ...
                ],
                "reason": "Combined evidence of tourism growth and local demand..."
            }

            The list is sorted highest score → lowest score.

    Notes:
        • No internal assumptions or external datasets are used.
        • All signals originate from Tavily search content.
        • Works globally for any country and any business type.
        • Serves as Tool 2 in the full location-selection pipeline.
    """

    ranked = []

    for city in candidate_cities:

        query = f"business demand foot traffic tourism startup activity for {business_type} in {city}, {country}. factual only."
        response = tavily.search(
            query=query,
            include_raw_content=True,
            max_results=7
        )

        enriched_data = []
        for r in response.get("results", []):
            enriched_data.append({
                "title": r.get("title"),
                "snippet": r.get("snippet"),
                "content": r.get("content"),
                "url": r.get("url")
            })

        # Extract factual signals from OpenAI
        extracted = extract_city_market_signals(city, business_type, enriched_data)
        signals = extracted.get("signals", [])
        reason = extracted.get("score_reason", "")

        # Score city
        score = score_city_from_signals(signals)

        ranked.append({
            "city": city,
            "score": score,
            "signals": signals,
            "reason": reason
        })

    # Sort by score (descending)
    ranked = sorted(ranked, key=lambda x: x["score"], reverse=True)

    return ranked

**Create a tool that generates the final business recommendation summary based strictly on the factual, already-extracted evidence from the ranked cities.**

In [ ]:
def generate_final_recommendation(country:str, business_type:str, ranked_cities:list[dict]) -> dict:
    """
    Generates the final business recommendation summary based strictly on the factual,
    already-extracted evidence from the ranked cities. This tool forms the final step
    of the decision pipeline (Tool #3), turning raw scored data into a clean,
    human-readable, minimalistic, and non-hallucinated recommendation.

    This tool does NOT introduce new facts, new interpretations, invented numbers,
    or speculative reasoning. It only restructures and compresses the data provided
    by `rank_cities_market_potential()`.

    Args:
        country (str):
            The country in which the business evaluation is taking place.
            Used to contextualize the generated title.

        business_type (str):
            The type of business being evaluated (e.g., "coffee shop", "bakery").
            Appears in both the title and the city summaries.

        ranked_cities (list[dict]):
            Output from `rank_cities_market_potential()`, already sorted in descending
            order of score. Each dictionary must contain:
                - "city": str
                - "score": int
                - "signals": list[str]
                - "reason": str

            This tool does not modify or reinterpret these values—it only summarizes them.

    Behavior:
        1. Selects the top 3 cities from the list (or fewer if less than 3 exist).
        2. Sends ONLY these top cities to OpenAI along with strict instructions:
            - No hallucination.
            - No new facts.
            - Title must be minimalistic (3–6 words).
            - Title must include country + business theme.
        3. LLM generates:
            - Clean title.
            - A list of cities with 1–2 sentence summaries each, derived solely from:
                • signals
                • reason
        4. If LLM output cannot be parsed as JSON, a safe fallback JSON structure is returned.

    Returns:
        dict:
            A structured JSON object with:
            {
              "title": "Minimalistic Title",
              "cities": [
                {
                  "city": "City Name",
                  "summary": "1–2 sentence factual summary based only on provided data."
                }
              ]
            }

            Title and summaries are always generated from the LLM but restricted
            to the facts extracted earlier in the pipeline.

    Notes:
        • This tool never performs new searches.
        • No new signals or metrics are introduced.
        • The summary is strictly a reformulation of existing extracted data.
        • This is Tool #3 in the complete pipeline:
                1. fetch_factual_location_data
                2. rank_cities_market_potential
                3. generate_final_recommendation  ← this function
    """

    top_cities = ranked_cities[:3] if len(ranked_cities) >= 3 else ranked_cities

    prompt = f"""
You are a factual recommendation summary generator.

GOAL:
Create a final recommendation summary for opening a **{business_type}** in **{country}**.

CONTENT RULES:
- USE ONLY the given data.
- DO NOT add new facts, numbers, or assumptions.
- DO NOT hallucinate.
- Title must be:
    • minimalistic
    • 3–6 words
    • includes country + business theme
    • Example: "Top US Coffee Locations"
- For each city, generate:
    • 1–2 sentence summary combining signals + reason
    • concise, business-oriented, factual
    • no invented claims
    • mention only what's present in the extracted signals & reason

DATA PROVIDED (must not extend beyond this):
{json.dumps(top_cities, indent=2, ensure_ascii=False)}

OUTPUT FORMAT (strict JSON):
{{
  "title": "string",
  "cities": [
    {{
      "city": "City Name",
      "summary": "1–2 sentence factual business summary."
    }}
  ]
}}
    """

    resp = openai_client.responses.create( model="gpt-4.1-mini", input=prompt, max_output_tokens=500 )

    try:
        return json.loads(resp.output_text)
    except:
        return {
            "title": f"{business_type.capitalize()} Opportunities in {country}",
            "cities": []
        }


**Test result** -> Output of the final recommendation summary

In [ ]:
generate_final_recommendation(country="France", business_type="bakery", ranked_cities=ranking)

{'title': 'France Bakery Opportunities',
 'cities': [{'city': 'Paris',
   'summary': "Paris has strong local and tourist demand for bakeries, supported by high foot traffic, culinary tourism, and government backing, though opening requires at least 10 years of baker experience. The city's bakery culture is embedded in daily life with significant online and travel brand interest."},
  {'city': 'Valbonne',
   'summary': 'Bakery tourism in Valbonne is growing, driven by social media and travelers seeking unique pastry experiences; there is strong consumer willingness to spend on high-quality craft bakery goods and participate in bakery workshops.'},
  {'city': 'Antibes',
   'summary': 'Antibes benefits from the global trend of bakery tourism, with tourists and locals prioritizing artisanal bakery experiences and quality products, reflecting a business opportunity for quality-focused bakeries.'}]}

**Define the interfaces for gradio**

In [ ]:
location_interface = gr.Interface(
    fn=fetch_factual_location_data,
    inputs=[
        gr.Textbox(label="Enter country", placeholder="Ex: Japan, USA"),
        gr.Textbox(label="Enter business", placeholder="Ex: Bakery, Fishing")
    ],
    outputs=gr.Textbox(label="Suitable cities for a business", lines=20),
    api_name="fetch_factual_location_data"
)

rank_city_interface = gr.Interface(
    fn=rank_cities_market_potential,
    inputs=[
        gr.Textbox(label="Enter country", placeholder="Ex: Japan, USA"),
        gr.Textbox(label="Enter business", placeholder="Ex: Bakery, Fishing")
    ],
    outputs=gr.Textbox(label="Ranked cities for a business", lines=30),
    api_name="rank_cities_market_potential_simple"
)

final_recommendation_interface = gr.Interface(
    fn=generate_final_recommendation,
    inputs=[
        gr.Textbox(label="Enter country", placeholder="Ex: Japan, USA"),
        gr.Textbox(label="Enter business", placeholder="Ex: Bakery, Fishing")
    ],
    outputs=gr.Textbox(label="Recommended cities for a business", lines=30),
    api_name="generate_final_recommendation_simple"
)

**Define UI in gradio and enable "mcp_server = True"**

In [ ]:
with gr.Blocks(title="Business Location Scouter: MCP server") as demo:
    gr.Markdown("""
        ## Business Location Scouter is an MCP server that identifies and ranks the best locations to open a business.
    """)

    gr.TabbedInterface(
        [
            location_interface,
            rank_city_interface,

            # Add more tools here
        ],
        [
            "Location_interface",
            "Rank_city_interface",
            # Add tool titles here

        ]

    )
    gr.Markdown("""
## 🎥 [Watch the Demo on YouTube](https://youtu.be/QjWTyAEODGA?si=OoEcizdhF5mtzRnB)
""")
    gr.Markdown("""
## **Example prompt to use on claude**: i want to open a bakery in sweden, can you first scout the places and then rank them and then based on that only give me top 3
""")

if __name__ == "__main__":
    demo.launch(mcp_server=True,share=True,theme=gr.themes.Citrus())